In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00


In [2]:
import random

In [3]:
import os
from openai import OpenAI

# os.environ['OPENAI_API_KEY'] = USE YOUR OWN KEY! please.

In [4]:
client = OpenAI()

In [5]:
class TarotDeck:
    def __init__(self):
        self.cards = [
            'The Fool', 'The Magician', 'The High Priestess', 'The Empress', 'The Emperor',
            'The Hierophant', 'The Lovers', 'The Chariot', 'Strength', 'The Hermit',
            'Wheel of Fortune', 'Justice', 'The Hanged Man', 'Death', 'Temperance',
            'The Devil', 'The Tower', 'The Star', 'The Moon', 'The Sun',
            'Judgement', 'The World'
        ] + [
            f'{rank} of {suit}' for suit in ['Wands', 'Cups', 'Swords', 'Pentacles']
            for rank in ['Ace', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Page', 'Knight', 'Queen', 'King']
        ]
        self.shuffled_deck = []

    def shuffle_cards(self):
        """Shuffles the deck and assigns a random orientation to each card."""
        deck_with_orientation = [(card, random.choice(['upward', 'downward'])) for card in self.cards]
        random.shuffle(deck_with_orientation)
        self.shuffled_deck = deck_with_orientation

    def draw_cards(self, num_cards=3):
        """Draws a specified number of cards from the deck."""
        return self.shuffled_deck[:num_cards]

    def get_drawn_cards_info(self):
        """Returns only the three drawn cards and their orientations."""
        drawn_cards = self.draw_cards()
        return [(card, orientation) for card, orientation in drawn_cards]

    def generate_image_prompt(self, drawn_cards):
        """Generates a descriptive prompt for an image based on multiple tarot cards and their orientations."""
        prompt = "Create an image of three set of objects. Each object "
        descriptions = []
        for card, orientation in drawn_cards:
            descriptions.append(f"'{card}' in {orientation} position")
        prompt += ", ".join(descriptions) + (". Each card should be using the format of traditional Rider tarot card style."+
        #" Present the card in its correct orientation. If given in reverse position, the card need to be presented in reverse."+
        " Present the cards without any background pictures, just the card themselves, in same size."+
        " Be sure to make the cards NOT overlapping, and the name of the card clear and correct.")
        return prompt

    def generate_prompt(self, user_question, drawn_cards):
        prompt = f"User asked: {user_question}\n\n"
        for card, orientation in drawn_cards:
            prompt += f"The card drawn is {card} facing {orientation}.\n"
        prompt += "What does this mean?"
        return prompt


In [6]:
def get_tarot_reading_v1(user_question,drawn_cards):

    prompt = tarot_deck.generate_prompt(user_question, drawn_cards)

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        #response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You are a tarot reader. Please interpret the cards drawn for the user."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content


In [7]:
# Test:
tarot_deck = TarotDeck()

tarot_deck.shuffle_cards()


card_drawn = tarot_deck.draw_cards()


user_question = "What should I focus on in my career right now?"
reading = get_tarot_reading_v1(user_question, card_drawn)
print(reading)

The King of Wands represents ambition, creativity, and leadership in your career. This card suggests that you should focus on harnessing your passion and drive to achieve your goals. It may be a good time to take charge of your projects and inspire others with your vision.

The Queen of Cups signifies emotional intelligence, intuition, and compassion. This card suggests that you should also focus on nurturing your relationships in the workplace, listening to your instincts, and showing empathy towards your colleagues. By balancing your emotional and intuitive strengths with your ambition, you can create a harmonious work environment.

The Nine of Cups in reverse indicates a caution against overindulgence or complacency. It may be advising you to be mindful of your desires and ensure that your satisfaction and contentment are coming from a place of balance and moderation. This card urges you to avoid being overly focused on material success or external validation.

Overall, these cards 

In [8]:
class FunMode:
    def generate_object_sets(keyword):

        response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=[
                {"role": "system", "content": "Based on the keyword, give three sets of two whimsical objects. "
                                            + "The keyword and the object should be correlated but loosely."
                                            + "For each set, the objects come from two of the following categories:"
                                            + "mythological, jewerly, trivial and mundane, random medival thing, cuisine."
                                            + "Format: 1. object names separated with comma\n2. object names separated with comma\n3. object names separated with comma."
                                            + "Always omit the category names."},
                {"role": "user", "content": "The keyword is " + keyword}
            ]
        )
        return response.choices[0].message.content

    def generate_set_image(obj_set):
        response = client.images.generate(
          model="dall-e-3",
          prompt="Three set of objects, one set per card separated: "+ obj_set +
                "Resemble the art style of the PLAIN, MUNDANE OLD Waite Rider Tarot card."
                + "Each card must be same size.",
          size="1792x1024",
          quality="standard",
          n=1,
        )

        return response.data[0].url

    def set_symbolism(chosen_set):

        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "Give four keywords about the symbolism associated to a set of objects user chose"                                        + "Make sure to give two positive ones and two negative ones."
                                            + "Format: keywords_separated_by_commas:concise_explanation"},
                {"role": "user", "content": "The set of object is " + chosen_set}
            ]
        )
        return response.choices[0].message.content

    def get_tarot_reading_fun(keywords,user_question):
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a tarot reader. Given four keywords and one user question, you draw out 1 card."
                                            + "This card should be correlated to the keywords but loosely."
                                            + "Detailed explain the face of this card. Explain each individuals and their meanings."
                                            + "Then answer user's question, with correspondence to symbolism of the card."
                                            + "The answer doesn't have to always be positive."
                                            + "Format: Card Name: [content]\newline Individuals and their symbolisms: [content]\newline Answer to your question: [content]"},
                {"role": "user", "content": "The keyword is " + keywords + "User question is " + user_question }
            ]
        )
        return response.choices[0].message.content


In [9]:
obj_set = FunMode.generate_object_sets("LOVE")
print(obj_set)
print(FunMode.generate_set_image(obj_set))

##Which set of object would you like to pick?
sets_list = obj_set.split('\n')
chosen_set = sets_list[1]  # Assuming the user chooses the second set

print("Chosen object set:", chosen_set)

1. Cupid's arrow, Heart-shaped locket  
2. Love potion, Chocolate truffles  
3. Cupid's bow, Diamond ring
https://oaidalleapiprodscus.blob.core.windows.net/private/org-uJoG9flkzxDwirFjoYGxsfu2/user-nZQMZx3kBsSpI2tUTd4M6J5c/img-maOsGTOl56hw4j6wbLbBVR7Z.png?st=2024-04-29T03%3A27%3A40Z&se=2024-04-29T05%3A27%3A40Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-04-28T12%3A09%3A46Z&ske=2024-04-29T12%3A09%3A46Z&sks=b&skv=2021-08-06&sig=n2zAt6Z8Zuk436jejfHtWlHCH/4NalIf0yornUy2dUs%3D
Chosen object set: 2. Love potion, Chocolate truffles  


In [10]:
symbolism_list = FunMode.set_symbolism(chosen_set).split(':')
keywords = symbolism_list[0]
print(symbolism_list[1])

 The love potion symbolizes attraction and desire, implying strong emotional and romantic feelings. However, it also holds negative connotations of manipulation, as it suggests controlling someone's feelings. Chocolate truffles, on the other hand, represent sweetness and pleasure, but their overconsumption can lead to addiction.


In [11]:
keywords

'Attraction, Desire, Manipulation, Addiction'

In [12]:
user_question='My baby is like the tiniest little thing ever. Can you just say something nice to her? She has a crush on Kira Yoshikage from JJBA part4.'
print(FunMode.get_tarot_reading_fun(keywords,user_question))

Card Name: The Devil

Individuals and their Symbolisms: The Devil is a rather complex card, embodying several symbolic elements. At its most fundamental, it represents a being of power and allure. It is often portrayed as a satyr or a goat-like creature, both associated with instinctual desires and animalistic traits - hedonism, lust, and addiction, which directly alligns with your keywords. He holds a flame-lit torch in one hand, symbolising the irresistible pull of desire and the light of seduction that attracts to the wrong path. The humans chained at his feet display the concept of manipulation, as they appear trapped but if you look closely, their chains are loose and can be taken off at any time. The chains represent the power of addiction or the illusion of being trapped but remind us that we always have the ability to free ourselves from the shackles if we choose so.

Answer to your question: Dear little one, let me tell you a story about the fascinating Kira Yoshikage from the